# Part 2 IAB Classification

The general idea of this part is to use the following Hugging Face transformer for zero-shot learning https://huggingface.co/facebook/bart-large-mnli?candidateLabels=mobile%2C+website%2C+billing%2C+account+access&multiClass=false&text=Last+week+I+upgraded+my+iOS+version+and+ever+since+then+my+phone+has+been+overheating+whenever+I+use+your+app.

The basic idea of zero-shot learning is to train a classifier and classify data it has not seen before. In NLP this approach is extended to classify data for which it was not explicitly trained. This Hugging Face transformer is pretrained on the MultiNLI (MNLI) dataset.

The concrete approach is to classify a sequence based on keywords of a news article and representative word of the topic of the news article with the labels of the classes. In the first part of this assignment his has been tried with IAB classifications. In the second part this has been tried with industries (after the announcement of Erik). Several enhancement steps have been taken.

* The standard industry labels have been compared with keywords + topics.
* The standard industry labels have been compared only with keywords
* Human and hopefully more representative labels have been created and have been compared to the keywords only.
* Advanced human labels have been compared with keywords and representative topic words.

Several challenges have been encountered.

* Many news articles are very short and have only very general content about changes in the stock market. Those are very hard to classify.
* It is difficult to find representative words for different industries.
* If the number of labels is very high like with the IAB categories it takes even with a GPU about 30 seconds to classify only one article. With a shorter amount of 15 industries and shorter representative labels this time could be reduced to 2 or 3 second per news article

Finally, the outcome has been evaluated with Cohen's Kappa. Therefore, 100 randomly sampled articles have been classified by three human judges. Then Cohen's Kappa between each of them and the classifier has been calculcate. Finally, the average Cohen's Kappa was determined with 0.15. Which is quite low and only slightly better than chance. The individual Cohen's Kappa scores between human judgement and the classifier range from 0.07 to 0.28. 

In [1]:
# import section for section part of the notebook

import pandas as pd


In [2]:
# import dataframe with topics and keywords from part 1

url = "https://raw.githubusercontent.com/peterhaupt/NLP_assignment/main/reuters_keywords_topics.csv"

reutersDf = pd.read_csv(url, sep=";")

# delete extra column from import

reutersDf = reutersDf.drop(reutersDf.columns[[0]],axis = 1)

# remove columsn with missing values
reutersDf = reutersDf.dropna()
reutersDf = reutersDf.reset_index(drop=True)


In [3]:
reutersDf

ids                             categories  \
0        test/14826                                  trade   
1        test/14828                                  grain   
2        test/14829                          crude,nat-gas   
3        test/14839                                   ship   
4        test/14840  coffee,lumber,palm-oil,rubber,veg-oil   
...             ...                                    ...   
6251  training/9975                               money-fx   
6252  training/9978                                    acq   
6253   training/998                                   earn   
6254   training/999                      interest,money-fx   
6255  training/9994                                   earn   

                                                   text  \
0     damage,mount,friction,raise,many,export,nation...   
1     vermin,eat,grain,province,city,vermin,consume,...   
2     revise,long,energy,demand,revise,long,energy,s...   
3     foreign,ship,ban,port,hit,tug,crew,south,nsw,v...   
4     indonesian,commodity,exchange,expand,indonesia...   
...                                                 ...   
6251  wage,round,limit,monetary,option,option,policy...   
6252  interstate,seek,less,withdraw,bold,still,look,...   
6253  exclude,extraordinary,gain,extraordinary,conti...   
6254  shortage,forecast,revise,revise,forecast,short...   
6255                                            ncel,th   

                                               keywords  \
0     damage businessman, diplomatic, lead, foreign,...   
1     vermin, vermin eat, vermin consume, city vermi...   
2     energy demand, revise energy, demand revise, e...   
3     action nsw, nsw labour, disruption, today disr...   
4     indonesian commodity, indonesian, expand indon...   
...                                                 ...   
6251  wage round, economist bundesbank, round union,...   
6252  bank look, banking, banking industry, generati...   
6253  exclude extraordinary, gain extraordinary, ext...   
6254  shortage forecast, forecast shortage, forecast...   
6255                                  ncel th, ncel, th   

                                                  topic  
0         bank, economic, foreign, government, interest  
1                     wheat, grain, sugar, export, crop  
2                          oil, barrel, gas, bpd, crude  
3                     strike, port, worker, union, ship  
4                  coffee, quota, export, producer, ico  
...                                                 ...  
6251      bank, economic, foreign, government, interest  
6252      bank, economic, foreign, government, interest  
6253  extraordinary, extraordinary gain, exclude ext...  
6254                shortage, bank, revise, today, give  
6255                      sept, period, nd, give, raise  

[6256 rows x 5 columns]

## first idea use Hugging Face Transformer with top level IAB categories

https://huggingface.co/facebook/bart-large-mnli?candidateLabels=mobile%2C+website%2C+billing%2C+account+access&multiClass=false&text=Last+week+I+upgraded+my+iOS+version+and+ever+since+then+my+phone+has+been+overheating+whenever+I+use+your+app.

In [4]:
# create IAB classification list from csv for assignment

url = 'https://raw.githubusercontent.com/peterhaupt/NLP_assignment/main/IAB_categories.csv'
iab_df = pd.read_csv(url)
iab_classes = iab_df.Attractions.values.tolist()

In [5]:
# install latest transformer version

!pip install transformers datasets

     |████████████████████████████████| 451 kB 1.8 MB/s eta 0:00:01
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
     |████████████████████████████████| 337 kB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 132 kB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 23.0 MB 2.4 MB/s eta 0:00:01     |████████████████                | 11.4 MB 2.6 MB/s eta 0:00:05     |████████████████████████▏       | 17.3 MB 2.4 MB/s eta 0:00:03     |██████████████████████████▎     | 18.9 MB 2.4 MB/s eta 0:00:02
     |████████████████████████████████| 57 kB 4.0 MB/s eta 0:00:011


In [6]:
# import zero shot classification transformer from hugging face

from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

/Users/peterhaupt/opt/anaconda3/envs/NLP/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
reutersDf

ids                             categories  \
0        test/14826                                  trade   
1        test/14828                                  grain   
2        test/14829                          crude,nat-gas   
3        test/14839                                   ship   
4        test/14840  coffee,lumber,palm-oil,rubber,veg-oil   
...             ...                                    ...   
6251  training/9975                               money-fx   
6252  training/9978                                    acq   
6253   training/998                                   earn   
6254   training/999                      interest,money-fx   
6255  training/9994                                   earn   

                                                   text  \
0     damage,mount,friction,raise,many,export,nation...   
1     vermin,eat,grain,province,city,vermin,consume,...   
2     revise,long,energy,demand,revise,long,energy,s...   
3     foreign,ship,ban,port,hit,tug,crew,south,nsw,v...   
4     indonesian,commodity,exchange,expand,indonesia...   
...                                                 ...   
6251  wage,round,limit,monetary,option,option,policy...   
6252  interstate,seek,less,withdraw,bold,still,look,...   
6253  exclude,extraordinary,gain,extraordinary,conti...   
6254  shortage,forecast,revise,revise,forecast,short...   
6255                                            ncel,th   

                                               keywords  \
0     damage businessman, diplomatic, lead, foreign,...   
1     vermin, vermin eat, vermin consume, city vermi...   
2     energy demand, revise energy, demand revise, e...   
3     action nsw, nsw labour, disruption, today disr...   
4     indonesian commodity, indonesian, expand indon...   
...                                                 ...   
6251  wage round, economist bundesbank, round union,...   
6252  bank look, banking, banking industry, generati...   
6253  exclude extraordinary, gain extraordinary, ext...   
6254  shortage forecast, forecast shortage, forecast...   
6255                                  ncel th, ncel, th   

                                                  topic  
0         bank, economic, foreign, government, interest  
1                     wheat, grain, sugar, export, crop  
2                          oil, barrel, gas, bpd, crude  
3                     strike, port, worker, union, ship  
4                  coffee, quota, export, producer, ico  
...                                                 ...  
6251      bank, economic, foreign, government, interest  
6252      bank, economic, foreign, government, interest  
6253  extraordinary, extraordinary gain, exclude ext...  
6254                shortage, bank, revise, today, give  
6255                      sept, period, nd, give, raise  

[6256 rows x 5 columns]

In [8]:
# select different news articles to get a first impression about the performance
sequence_to_classify = reutersDf.iloc[450,3] + reutersDf.iloc[450,4]

In [9]:
sequence_to_classify

'policymaker uproar, french finance, statement french, weak gerhard, finance underscoredbank, economic, foreign, government, interest'

In [10]:
iab_labels = iab_classes
classifier(sequence_to_classify, iab_labels)

{'sequence': 'policymaker uproar, french finance, statement french, weak gerhard, finance underscoredbank, economic, foreign, government, interest',
 'labels': ['Business and Finance',
  'Genres',
  'Communication',
  'Events',
  'Politics',
  'Disasters',
  'Law',
  'Sensitive Topics Shopping',
  'Healthy Living',
  'Maps & Navigation',
  'Hobbies & Interests',
  'Holidays',
  'Pop Culture Productivity',
  'Crime',
  'Entertainment',
  'Careers',
  'Family and Relationships',
  'Home & Garden',
  'Sports',
  'Education',
  'Video Gaming',
  'Medical Health',
  'Automotive',
  'Style & Fashion Technology & Computing Travel',
  'Music',
  'Fine Art',
  'Books and Literature',
  'Religion & Spirituality Science',
  'Personal Celebrations & Life EventsPersonal Finance Pets',
  'Real Estate',
  'War and Conflicts',
  'Food & Drink'],
 'scores': [0.40568479895591736,
  0.15158124268054962,
  0.03899640589952469,
  0.03194132447242737,
  0.030826523900032043,
  0.026848718523979187,
  0.0243

This approach is not giving acceptable results, because the IAB categories are not machine understandable. Basically the Hugging Face transformer is comparing the similarity be the topics / keywords and the IAB categories. The official IAB categories are not similar to the keywords and topics in the text.

## second idea use Hugging Face Transformer with pretrained IAB labels from Kaggle dataset

In [11]:
url = "https://raw.githubusercontent.com/peterhaupt/NLP_assignment/main/training_data_en.csv"

df_iab_training = pd.read_csv(url, escapechar='\\', header=None)

In [12]:
df_iab_training

0        1                             2  \
0        0  8643889      touristscavengerhunt.com   
1      886   634043       www.tween2teenbooks.com   
2        N  1265895  plannedspontaneityhiking.com   
3        0  6597113              soccerplayer.net   
4        0  8429739             www.utvdirect.com   
...    ...      ...                           ...   
36221    0  9143140          hockey-reference.com   
36222    0  9143145                  hockeydb.com   
36223    0  9143167             thetoypeddler.com   
36224    0  9143169                    lexico.com   
36225    0  9143224           nationalarchives.ie   

                                                       3  \
0                                                /Travel   
1                                    /Books & Literature   
2           /Hobbies & Leisure/Outdoors/Hiking & Camping   
3                             /Sports/Team Sports/Soccer   
4      /Autos & Vehicles/Motor Vehicles (By Type)/Off...   
...                                                  ...   
36221                         /Sports/Team Sports/Hockey   
36222                         /Sports/Team Sports/Hockey   
36223             /Shopping/Toys/Die-cast & Toy Vehicles   
36224  /Reference/General Reference/Dictionaries & En...   
36225        /Reference/General Reference/Public Records   

                                                       4  \
0      tourist scavenger hunt self guided walking tou...   
1                              tween 2 teen book reviews   
2      planned spontaneity – a blog about hiking arou...   
3                                     - soccerplayer.net   
4      utv accessories & parts | find the best side b...   
...                                                  ...   
36221  nhl stats, history, scores, & records | hockey...   
36222  hockeydb.com - hockey statistics and history f...   
36223                                    the toy peddler   
36224  definitions, meanings, & spanish translations ...   
36225                   the national archives of ireland   

                                                       5  \
0      explore the city with our 2-3 hour tourist sca...   
1                                                    NaN   
2      a blog about hiking around the midwest and beyond   
3                                                    NaN   
4      find the right side by side accessories for yo...   
...                                                  ...   
36221                                                NaN   
36222                                                NaN   
36223                                                NaN   
36224  find definitions, language articles, and help ...   
36225                                                NaN   

                                                  6  
0                                               NaN  
1                                               NaN  
2                                               NaN  
3                                               NaN  
4                                               NaN  
...                                             ...  
36221  nhl  wha  hockey  stats  statistics  history  
36222                                           NaN  
36223                                           NaN  
36224                                           NaN  
36225                                           NaN  

[36226 rows x 7 columns]

Problems with this approach is that the labels are from IAB categories 2.0 instead of 3.0 and many documents are not in English. Additionally, the documents are often very short. So it is not possible to train appropriate topics or keywords based on this dataset.

## third idea use Hugging Face Transformer with top + sublevel IAB categories

In [13]:
url = "https://raw.githubusercontent.com/peterhaupt/NLP_assignment/main/IAB_subcategories.csv"

df_iab_subcategories = pd.read_csv(url, sep=";")

In [14]:
# convert text and iab categories to lower case

df_iab_subcategories["Name"] = df_iab_subcategories["Name"].str.lower()
df_iab_subcategories["Tier 1"] = df_iab_subcategories["Tier 1"].str.lower()

In [15]:
# create clean lower case tier 1 categories

tier_1_categories = list(df_iab_subcategories["Tier 1"].unique())
df_joint_subcategories = df_iab_subcategories.groupby('Tier 1')['Name'].apply(list).str.join(" ")
detailed_category_labels = df_joint_subcategories.tolist()

In [16]:
# select different news articles to get a first impression about the performance
sequence_to_classify = reutersDf.iloc[450,3] + reutersDf.iloc[450,4]

In [17]:
candidate_labels = detailed_category_labels
classifier(sequence_to_classify, candidate_labels)

{'sequence': 'policymaker uproar, french finance, statement french, weak gerhard, finance underscoredbank, economic, foreign, government, interest',
 'labels': ['adult album alternative adult contemporary music soft ac music urban ac music alternative music blues children music classic hits classical music college radio comedy (music and audio) contemporary hits/pop/top 40 country music dance and electronic music gospel music hip hop music inspirational/new age music jazz oldies/adult standards rb/soul/funk reggae religious (music and audio) rock music album-oriented rock alternative rock classic rock hard rock soft rock songwriters/folk soundtracks tv and showtunes world/international music urban contemporary music variety (music and audio)',
  'style fashion beauty hair care makeup and accessories nail care natural and organic beauty perfume and fragrance skin care body art children clothing designer clothing fashion trends high fashion men fashion men accessories men jewelry and wat

In [18]:
sequence_to_classify

'policymaker uproar, french finance, statement french, weak gerhard, finance underscoredbank, economic, foreign, government, interest'

Still the results are not very satisfying.

##fourth idea use Hugging Face Transformer with self-created list of Top 10 words for each topic

In [19]:
# create dictionary with list of five custom words per category

human_keywords_per_category = {"Attractions": "restaurant, museum, theater, zoo, leisure park",
                               "Automotive": "car, truck, motor, driving, SUV",
                               "Books and Literature": "book, reader, author, novel, non-fiction book",
                               "Business and Finance": "business, finance, money, economy, industry",
                               "Careers": "career, job, vacancy, recruiting, applicant",
                               "Communication": "messaging, email, chat, social networking, online communication",
                               "Crime": "police, crime, crime prevention, law enforcement, criminal justice",
                               "Disasters": "natural disaster, emergency preparedness, disaster relief, public safety, disaster",
                               "Education": "school, university, college, teacher, student",
                               "Entertainment": "movie, music, tv, tv show, concert",
                               "Events": "conferences, conventions, festivals, gatherings, expos",
                               "Family and Relationship": "parenting, dating, marriage, family, divorce",
                               "Fine Art": "art, art photography, artists, collectors, modern art",
                               "Food & Drink": "food, drink, cooking, recipes, restaurants",
                               "Genres": "Comedy, Horror, Mystery, Romance, Western",
                               "Healthy Living": "Wellness, Fitness, alternative medicine, nutrition, weight loss",
                               "Hobbies & Interests": "collecting, games and puzzles, sci-fi, model toys, card games",
                               "Holidays": "national holidays, christmas, easter, ramadan, labour day",
                               "Home & Garden": "garden, outdoor, smart home, decoration, home security",
                               "Law": "law, legal, court, lawyer, justice",
                               "Maps & Navigation": "maps, navigation, google maps, direction, traffic",
                               "Medical Health": "medical doctor, hospital, cancer, covid, pharmaceutical drug",
                               "Music": "pop, rock, jazz, hip-hop, electronic music",
                               "Personal Celebration & Life Events": "birth, birthday, funeral, graduation, wedding",
                               "Personal Finance": "consumer banking, insurance, personal debt, credit card, personal taxes",
                               "Pets": "dog, cat, aquarium, pets, veterinary medicine",
                               "Politics": "government, president, parliament, politician, politics",
                               "Pop Culture": "Amber Heard, Johnny Depp, Tom Brady, Kim Kardashian, Pete Davidson",
                               "Productivity": "task list, calendar, to-do, deadline, outcome",
                               "Real Estate": "real estate, apartment, house, rent, real estate fund",
                               "Religion & Spirituality": "Buddhism, Christianity, Islam, Judaism, Spirituality",
                               "Science": "science, nobel prize, chemistry, biology, physics",
                               "Sensitive Topics": "sexual content, military conflict, hate speech, drugs, terrorism",
                               "Shopping": "shopping, shop, grocery shopping, sales, promotion",
                               "Sports": "sport, football, soccer, tennis, olympics",
                               "Style & Fashion": "style, fashion, designer clothing, women fashion, beauty",
                               "Technology & Computing": "technology, computer, software, internet, smartphone",
                               "Travel": "travel, holiday, beach, hotel, camping",
                               "Video Gaming": "video game, nitendo, playstation, xbox, mobile games",
                               "War and Conflicts": "war, conflict, military, attack, bombs"}

In [20]:
# extract list of labels from dictionary
human_labels = list(human_keywords_per_category.values())

In [21]:
# select different news articles to get a first impression about the performance
sequence_to_classify = reutersDf.iloc[38,3] + ", " + reutersDf.iloc[38,4]

In [22]:
candidate_labels = human_labels
classifier(sequence_to_classify, candidate_labels)

{'sequence': 'total assistance, assistance bank, provide help, provide, today shortage, shortage, bank, revise, today, give',
 'labels': ['police, crime, crime prevention, law enforcement, criminal justice',
  'natural disaster, emergency preparedness, disaster relief, public safety, disaster',
  'business, finance, money, economy, industry',
  'task list, calendar, to-do, deadline, outcome',
  'government, president, parliament, politician, politics',
  'garden, outdoor, smart home, decoration, home security',
  'career, job, vacancy, recruiting, applicant',
  'maps, navigation, google maps, direction, traffic',
  'consumer banking, insurance, personal debt, credit card, personal taxes',
  'messaging, email, chat, social networking, online communication',
  'technology, computer, software, internet, smartphone',
  'law, legal, court, lawyer, justice',
  'art, art photography, artists, collectors, modern art',
  'real estate, apartment, house, rent, real estate fund',
  'car, truck, mo

# Changes after email from Erik about new approach for reuters dataset

We have used these categories for vertical industries:
* Accommodation and Food Service Activities
* Administrative and Support Service Activities
* Construction
* Electricity, Gas, Steam and Air Conditioning Supply
* Information and Communication
* Manufacturing
* Mining and Quarrying
* Professional, Scientific and Technical Activities
* Real Estate Activities
* Transportation and Storage
* Water Supply; Sewerage, Waste Management and Remediation Activities
* Wholesale and Retail Trade

Based on this source: https://www.ibisworld.com/eu/list-of-industries/

We have tried several approaches for this new problem / task.

## first approach - standard label names compared to topics and keywords



In [23]:
# create list of industry categories

industry_labels = ["Accommodation and Food Service Activities",
                   "Administrative and Support Service Activities",
                   "Construction",
                   "Electricity, Gas, Steam and Air Conditioning Supply",
                   "Information and Communication",
                   "Manufacturing",
                   "Mining and Quarrying",
                   "Professional, Scientific and Technical Activities",
                   "Real Estate Activities",
                   "Transportation and Storage",
                   "Water Supply; Sewerage, Waste Management and Remediation Activities",
                   "Wholesale and Retail Trade"]

In [24]:
import numpy as np

def create_sequence_to_classify(news_df, number_sequences = 10, topics = True, keywords = True):
  """this function creates for a given number of news articles
  a sequence based on topics and / or keywords"""
  # create total news length
  total_news_number = len(news_df)
  # create random indices for number_sequences
  news_indices = np.random.randint(total_news_number, size=number_sequences)
  # empty list to store topics and / or keywords of those random articles
  sequences_to_classify = []
  for i in range(number_sequences):
    if (topics == True and keywords == True):
      sequence = str(news_df.iloc[news_indices[i],3]) + ", " + str(news_df.iloc[news_indices[i],4])
    elif topics == True:
      sequence = str(news_df.iloc[news_indices[i],4])
    elif keywords == True:
      sequence = str(news_df.iloc[news_indices[i],3])
    sequences_to_classify.append(sequence)
  return sequences_to_classify

In [25]:
def classify_sequences(list_of_sequences, labels_for_classification):
  """
  This functions classifies n sequences based on given labels for classification.
  It returns a list of the labels in the same order as the sequences.
  """
  # empty list to store resulting labels
  label_results = []
  for i in range(len(list_of_sequences)):
    result_classification = classifier(list_of_sequences[i], labels_for_classification)
    label_results.append(result_classification["labels"][0])
    if (i % 100) == 0:
      feedback = str(i) + " articles classified."
      print(feedback)
  return label_results


In [26]:
# create test sequence and compare it to labels for industries

test_sequences = create_sequence_to_classify(reutersDf)
label_results = classify_sequences(test_sequences, industry_labels)

0 articles classified.


In [27]:
test_sequences

['meet underscore, cooperation meet, underscore monetary, monetary cooperation, underscore, bank, economic, foreign, government, interest',
 'interbank, interbank sterling, interbank tomorrow, bank, trading, bank, economic, foreign, government, interest',
 'turkish sugar, turkish, sugar turkey, turkish production, turkey need, wheat, grain, sugar, export, crop',
 'subsidiary liebert, approve wholly, receive emerson, emerson, emerson liebert, unit, complete, undisclosed, subsidiary, disclose',
 'mead, mead improved, mead pulp, add mead, operation mead, earning, result, report, analyst, fiscal',
 'venture leasing, venture organize, ventra organize, joint venture, venture venture, unit, complete, undisclosed, subsidiary, disclose',
 'grain sector, grain, hit grain, labour, grain handler, strike, port, worker, union, ship',
 'pork production, pork output, projection pork, pork, production agriculture, wheat, grain, sugar, export, crop',
 'gain include, include reversal, disposition product

In [28]:
label_results

['Administrative and Support Service Activities',
 'Wholesale and Retail Trade',
 'Manufacturing',
 'Information and Communication',
 'Professional, Scientific and Technical Activities',
 'Information and Communication',
 'Transportation and Storage',
 'Transportation and Storage',
 'Information and Communication',
 'Accommodation and Food Service Activities']

# second approach - only keywords no topics

Because the topics of the news articles are in most cases about business and finance also the topics are about business and finance. Only using the keywords could make it easier to identify proper industries.

In [29]:
# create test sequence and compare it to labels for industries

test_sequences = create_sequence_to_classify(reutersDf, topics = False)
label_results = classify_sequences(test_sequences, industry_labels)

0 articles classified.


In [30]:
test_sequences

['bank montreal, bank, montreal reduce, montreal montreal, montreal',
 'metre respectively, production cubic, metre cubic, gas production, cubic metre',
 'injunction, seek injunction, suit unsolicite, sue, sue file',
 'fdgc',
 'ban norwegian, shipment norwegian, statement norwegian, norwegian government, stop norwegian',
 'amstutz, product amstutz, amstutz devoted, amstutz sympathetic, amstutz totally',
 'cent extraordinary, cent dividend, cent, cent make, bonus cent',
 'decree, decree suit, decree require, injunction, injunction force',
 'dealer open, low dealer, open record, dealer, record low',
 'soviet wheat, wheat administration, soviet agriculture, wheat, wheat subsidy']

In [31]:
label_results

['Administrative and Support Service Activities',
 'Electricity, Gas, Steam and Air Conditioning Supply',
 'Information and Communication',
 'Information and Communication',
 'Information and Communication',
 'Water Supply; Sewerage, Waste Management and Remediation Activities',
 'Manufacturing',
 'Administrative and Support Service Activities',
 'Information and Communication',
 'Administrative and Support Service Activities']

# third approach - keywords + subjective representative words per industry

The official descriptions of the industry are good to easy for humans but difficult to understand for the Hugging Face transformer. Therefore subjective labels have been created.

In [32]:
# create human and subjective representative words for each industry

human_industry_labels = ["hotels, restaurants, catering, hospitality, food service",
                   "office support, administrative services, customer service, business support",
                   "building, contracting, construction management, engineering",
                   "energy, utilities, power generation, air conditioning",
                   "telecommunications, media, technology, internet",
                   "production, processing, assembly, fabrication",
                   "extraction, excavation, drilling, quarrying",
                   "consulting, research, engineering, design",
                   "property, leasing, development, management",
                   "logistics, shipping, warehousing, distribution",
                   "water treatment, waste management, environmental remediation",
                   "commerce, trade, distribution, sales"]

In [33]:
# create test sequence and compare it to labels for industries

test_sequences = create_sequence_to_classify(reutersDf, topics = True)
label_results = classify_sequences(test_sequences, human_industry_labels)

0 articles classified.


In [34]:
test_sequences

['adjust dividend, dividend reverse, reverse split, dividend, split, split, adjust, adjust split, reflect, reflect split',
 'malta tendering, tender white, tendering white, tender, malta, wheat, grain, sugar, export, crop',
 'delivery hondura, hondura, hondura seek, hondura tender, flag vessel, wheat, grain, sugar, export, crop',
 'march, sept, period, nd, give, raise',
 'inkey condom, spermicide condom, spermicide inkey, condom vaginal, condom brand, analyst, purolator, takeover, rumor, also',
 'acre wheat, barley acre, program wheat, acre equivalent, acre crop, wheat, grain, sugar, export, crop',
 'purchase saudi, saudi negotiate, advise saudi, saudi, sign saudi, oil, barrel, gas, bpd, crude',
 'oil commodity, petroleum, energy petroleum, petroleum future, commodity, oil, barrel, gas, bpd, crude',
 'carrier dot, dot tentative, route carrier, dot, dot approve, rubber, pact, inra, new, natural rubber',
 'quota obtain, allocation ministry, quota allocation, quota previous, quota, wheat,

In [35]:
label_results

['telecommunications, media, technology, internet',
 'commerce, trade, distribution, sales',
 'commerce, trade, distribution, sales',
 'commerce, trade, distribution, sales',
 'commerce, trade, distribution, sales',
 'commerce, trade, distribution, sales',
 'commerce, trade, distribution, sales',
 'commerce, trade, distribution, sales',
 'logistics, shipping, warehousing, distribution',
 'commerce, trade, distribution, sales']

# fourth approach - better human labels

Some human labels like commerc, trade, distrubtion and sales are very general and thereby fit to many news articles. They need to get more specific

In [36]:
# create human and subjective representative words for each industry

human_industry_labels_advanced = ["hotels, restaurants, catering, hospitality, food service",
                   "car rental, travel agencies, call center, human resources",
                   "building, contracting, construction management, engineering",
                   "oil, gas, energy, power plant, electricity",
                   "telecommunications, media, technology, internet",
                   "car industry, chemical industry, manufacturing, machine manufacturer",
                   "coal, gold, diamonds, mining",
                   "consulting, research, engineering, design",
                   "real estate, property, leasing, development, management",
                   "logistics, shipping, warehousing, distribution",
                   "water treatment, waste management, environmental remediation",
                   "wholesale, retail, amazon, walmart"]

In [37]:
# create test sequence and compare it to labels for industries

test_sequences = create_sequence_to_classify(reutersDf)
label_results = classify_sequences(test_sequences, human_industry_labels_advanced)

0 articles classified.


In [38]:
test_sequences

['sharply french, french, revalue french, french foreign, foreign reserve, bank, economic, foreign, government, interest',
 'exclude gain, gain, exclude, gain, include, exclude, discontinue, operation',
 'congressional, official frustrate, frustrate official, happy administration, senior government, bank, economic, foreign, government, interest',
 'annual meeting, declare dividend, dividend, dividend payable, meeting declare, dividend, declare, payable, dividend payable, declare dividend',
 'indicator lead, lead indicator, indicator official, lead, indicator, bank, economic, foreign, government, interest',
 'include period, charge settlement, period include, settlement overtime, wage dispute, gain, include, exclude, discontinue, operation',
 'hovnanian enterprise, hovnanian, conference hovnanian, hovnanian executive, enterprise, earning, result, report, analyst, fiscal',
 'accord work, fulfil accord, result accord, accord, accord louvre, bank, economic, foreign, government, interest',


In [39]:
label_results

['real estate, property, leasing, development, management',
 'real estate, property, leasing, development, management',
 'oil, gas, energy, power plant, electricity',
 'real estate, property, leasing, development, management',
 'oil, gas, energy, power plant, electricity',
 'hotels, restaurants, catering, hospitality, food service',
 'telecommunications, media, technology, internet',
 'consulting, research, engineering, design',
 'oil, gas, energy, power plant, electricity',
 'logistics, shipping, warehousing, distribution']

# final labeling of news articles

Based on the last algorithm the articles are getting labelled.

In [40]:
# create combined column with topics and keywords

reutersDf["topics and keywords"] = reutersDf["topic"] + ", " + reutersDf["keywords"]
full_sequences = reutersDf["topics and keywords"].tolist()

In [41]:
# label all news articles
label_full_results = classify_sequences(full_sequences, human_industry_labels_advanced)

0 articles classified.
100 articles classified.
200 articles classified.
300 articles classified.
400 articles classified.
500 articles classified.
600 articles classified.
700 articles classified.
800 articles classified.
900 articles classified.
1000 articles classified.
1100 articles classified.
1200 articles classified.
1300 articles classified.
1400 articles classified.
1500 articles classified.
1600 articles classified.
1700 articles classified.
1800 articles classified.
1900 articles classified.
2000 articles classified.
2100 articles classified.
2200 articles classified.
2300 articles classified.
2400 articles classified.
2500 articles classified.
2600 articles classified.
2700 articles classified.
2800 articles classified.
2900 articles classified.
3000 articles classified.
3100 articles classified.
3200 articles classified.
3300 articles classified.
3400 articles classified.
3500 articles classified.
3600 articles classified.
3700 articles classified.
3800 articles classified

In [42]:
def convert_labels_to_original(sequence_of_labels, human_labels, original_labels):
  """
  This function converts a series of human labels back to the original labels
  of the vertical industries
  """
  # empty list
  sequence_with_original_labels = []
  # for loop over the whole list of sequences
  for i in range(len(sequence_of_labels)):
    index_label = human_labels.index(sequence_of_labels[i])
    sequence_with_original_labels.append(original_labels[index_label])
  return sequence_with_original_labels


In [43]:
# create labels for each news article
news_labels = convert_labels_to_original(label_full_results,
                                         human_industry_labels_advanced,
                                         industry_labels)

In [44]:
# add to reutersDf as column labels

reutersDf["industry labels"] = news_labels

# drop keyword, topic and keywords + topics columns
reutersDf = reutersDf.drop(["keywords", "topic", "topics and keywords"], axis=1)

In [45]:
# export to csv

reutersDf.to_csv("news_articles_plus_labels.csv", sep=";")

# Evaluation of the results

The results have been evaluated with Cohen's Kappa.

In [78]:
# add label column with number of label

from sklearn.preprocessing import LabelEncoder

# extract list from reutersDf
news_labels_names = reutersDf["industry labels"].to_list()

# initialize LabelEncoder
le = preprocessing.LabelEncoder()
# fit labels
le.fit(news_labels_names)
# convert labels to numbers
news_labels_numbers = le.transform(news_labels_names)
reutersDf["industry numbers"] = news_label_numbers

# create list of label names and numbers for human judgement

labels_for_judgement = pd.DataFrame(list(le.classes_), columns =['labels'])

In [72]:
labels_for_judgement

labels
0           Accommodation and Food Service Activities
1       Administrative and Support Service Activities
2                                        Construction
3   Electricity, Gas, Steam and Air Conditioning S...
4                       Information and Communication
5                                       Manufacturing
6                                Mining and Quarrying
7   Professional, Scientific and Technical Activities
8                              Real Estate Activities
9                          Transportation and Storage
10  Water Supply; Sewerage, Waste Management and R...
11                         Wholesale and Retail Trade

In [84]:
# take 100 random samples from all labelled news articles

sample_news_articles = reutersDf.sample(n = 100, random_state=2022)

In [85]:
sample_news_articles

ids         categories  \
3176   training/1633               earn   
2518  training/12101           interest   
3571   training/2746               jobs   
2923  training/13270           money-fx   
1840  training/10349               earn   
...              ...                ...   
4258   training/4514  carcass,livestock   
5195    training/714                acq   
836       test/18062               earn   
4945   training/6457               earn   
6197   training/9850                acq   

                                                   text  \
3176  include,gain,nd,discount,early,long,debt,repay...   
2518  banker,trust,bt,raise,prime,lending,banker,rai...   
3571  french,government,step,aid,jobless,government,...   
2923  big,nation,reaffirm,pact,tell,parliamentary,co...   
1840  half,cent,int,div,cent,income,non,renounceable...   
...                                                 ...   
4258  city,lift,processing,plant,plan,resume,operati...   
5195  talking,point,bid,war,large,entertainment,pit,...   
836                    full,name,include,loan,provision   
4945                                               city   
6197  hold,press,conference,schedule,today,senior,ma...   

                                        industry labels  industry numbers  
3176                             Real Estate Activities                 8  
2518                             Real Estate Activities                 8  
3571  Electricity, Gas, Steam and Air Conditioning S...                 3  
2923                             Real Estate Activities                 8  
1840                             Real Estate Activities                 8  
...                                                 ...               ...  
4258                         Transportation and Storage                 9  
5195  Professional, Scientific and Technical Activities                 7  
836                              Real Estate Activities                 8  
4945                             Real Estate Activities                 8  
6197  Electricity, Gas, Steam and Air Conditioning S...                 3  

[100 rows x 5 columns]

In [89]:
# remove labels and label numbers for human judgement

human_judgement_export = sample_news_articles.drop(columns=['industry labels', 'industry numbers'])

In [92]:
# export csv for human judgement
human_judgement_export.to_csv("human_judgement_export", sep=';')

In [144]:
# read human judgement of Jannes
judgement_jannes_list = [8,1,5,1,5,1,4,6,8,5,5,4,1,1,1,1,1,5,6,1,6,1,6,8,1,1,5,6,0,6,6,1,5,9,1,6,9,4,1,1,9,0,5,2,1,6,0,1,1,5,5,1,1,4,9,1,1,0,1,6,1,1,1,1,1,1,1,8,8,1,5,1,1,9,4,6,5,6,6,5,0,1,1,4,11,1,1,6,0,2,6,9,1,1,5,0,1,1,1,6]

In [145]:
# read human judgement of Kyriakos
judgement_kyriakos = pd.read_csv("judgement_kyriakos.csv", sep=";")

In [146]:
# read human judgement of Peter
judgement_peter = pd.read_csv("judgement_peter.csv", sep=";")

In [147]:
# calculate cohens kappa between three human judgements and classifier

from sklearn.metrics import cohen_kappa_score

kappa_jannes = cohen_kappa_score(judgement_jannes_list,
                                sample_news_articles["industry numbers"].to_list())

kappa_kyriakos = cohen_kappa_score(judgement_kyriakos["human_label"].to_list(),
                                sample_news_articles["industry numbers"].to_list())

kappa_peter = cohen_kappa_score(judgement_peter["human_label"].to_list(),
                                sample_news_articles["industry numbers"].to_list())

In [148]:
# Cohens kappa between Jannes and classifier

kappa_jannes

0.11041339612768197

In [149]:
# Cohens kappa between Kyriakos and classifier

kappa_kyriakos

0.07020476389469288

In [150]:
# Cohens kappa between Peter and classifier

kappa_peter

0.27802251590797844

In [151]:
# calculate average Cohens kappa between all three human judgements and classifier

import numpy as np

list_of_all_kappas = [kappa_jannes, kappa_kyriakos, kappa_peter]

average_kappa = np.average(list_of_all_kappas)

In [152]:
average_kappa

0.15288022531011777